# powerpoint 켜서 텍스트 박스 만들고 텍스트 입력하기.

In [1]:
import win32com.client

# 파워포인트 켜기.
App = win32com.client.Dispatch("PowerPoint.Application")
App.Visible = True

# 창 활성화.
App.Activate

# presentation 시작하고
prs = App.Presentations.Add(-1)
#sld = App.ActiveWindow.View.Slide
# 첫번째 슬라이드 활성화.
sld = prs.Slides.Add(1,12) # 12:ppLayoutBlank

# 도형추가.
shp = sld.Shapes.AddShape(1, 100, 100, 200, 100)

# 그 도형에 텍스트 추가.
shp.TextFrame.TextRange.Text = "안녕하세요 저는 정규단입니다."

# 그 도형 색깔 바꾸기.
shp.Fill.ForeColor.RGB = 42495 #'42495:rgbOrange
 
# msgbox
# shl = win32com.client.Dispatch("wscript.shell")
# shl.Popup ("Hello, world!")


### 이미 있는 텍스트 수정

In [2]:
import win32com.client

# 이미 실행 중인 PowerPoint 인스턴스에 연결
App = win32com.client.Dispatch("PowerPoint.Application")
App.Visible = True

# 현재 활성화된 프레젠테이션 가져오기
presentation = App.ActivePresentation

# 현재 활성화된 슬라이드 가져오기
active_slide = App.ActiveWindow.View.Slide

# 특정 텍스트가 포함된 텍스트 박스만 수정
for shape_idx in range(1, active_slide.Shapes.Count + 1):
    shape = active_slide.Shapes.Item(shape_idx)
    
    if shape.HasTextFrame and shape.TextFrame.HasText:
        if "안녕하세요" in shape.TextFrame.TextRange.Text:
            shape.TextFrame.TextRange.Text = "안녕하세요, 수정되었습니다!"

### decorater 이용 delay

In [4]:
import win32com.client
import time
import functools

def with_delay(delay=0.5):
    """
    함수 실행 후 지정된 시간만큼 대기하는 데코레이터
    """
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            result = func(*args, **kwargs)
            time.sleep(delay)
            return result
        return wrapper
    return decorator

# 기본 동작들을 데코레이터로 감싼 함수로 정의
@with_delay()
def create_ppt_app():
    app = win32com.client.Dispatch("PowerPoint.Application")
    app.Visible = True
    return app

@with_delay()
def create_presentation(app):
    return app.Presentations.Add(-1)

@with_delay()
def add_slide(presentation):
    return presentation.Slides.Add(1, 12)  # 12:ppLayoutBlank

@with_delay()
def add_shape(slide):
    return slide.Shapes.AddShape(1, 100, 100, 200, 100)

@with_delay()
def set_text(shape, text):
    shape.TextFrame.TextRange.Text = text

@with_delay()
def set_color(shape, color):
    shape.Fill.ForeColor.RGB = color

# 실행 코드
app = create_ppt_app()
app.Activate()  # 필요하면 이것도 데코레이터로 감쌀 수 있음
prs = create_presentation(app)
sld = add_slide(prs)
shp = add_shape(sld)
set_text(shp, "안녕하세요 저는 정규단입니다.")
set_color(shp, 42495)  # 42495:rgbOrange

### 단순 delay

In [17]:
import win32com.client
import time

# 진행상황 보이게.
def animated_execution(func, delay=0.5):
    result = func()
    time.sleep(delay)
    return result

# 파워포인트 켜기
App = animated_execution(lambda: win32com.client.Dispatch("PowerPoint.Application"))
animated_execution(lambda: setattr(App, 'Visible', True))

# 창 활성화
animated_execution(lambda: App.Activate())

# presentation 시작
prs = animated_execution(lambda: App.Presentations.Add(-1))

# 첫번째 슬라이드 활성화
sld = animated_execution(lambda: prs.Slides.Add(1, 12))  # 12:ppLayoutBlank

# 도형추가
shp = animated_execution(lambda: sld.Shapes.AddShape(1, 100, 100, 200, 100))

# 그 도형에 텍스트 추가
animated_execution(lambda: setattr(shp.TextFrame.TextRange, 'Text', "안녕하세요 저는 정규단입니다."))

# 그 도형 색깔 바꾸기
animated_execution(lambda: setattr(shp.Fill.ForeColor, 'RGB', 42495))  # 42495:rgbOrange

### 6개 슬라이드가 있을 때 1번부터 5번 슬라이드까지의 제목을 빨간색으로 바꾸기.

In [16]:
import win32com.client

# 이미 실행 중인 PowerPoint 애플리케이션에 연결
App = win32com.client.GetObject(Class="PowerPoint.Application")

# 현재 활성화된 프레젠테이션 가져오기
presentation = App.ActivePresentation

# RGB 색상 값 정의 (파란색: R=0, G=0, B=255)
blue_color_rgb = 25  # 파란색의 RGB 값 (win32com에서는 RGB 순서가 반대)

# 1번부터 5번 슬라이드까지 반복
for slide_index in range(1, 6):
    try:
        # 슬라이드 가져오기 (PowerPoint는 1부터 인덱싱)
        slide = presentation.Slides(slide_index)
        
        # 슬라이드의 모든 도형 순회
        for shape_index in range(1, slide.Shapes.Count + 1):
            shape = slide.Shapes(shape_index)
            
            # 제목 플레이스홀더인지 확인 (PlaceholderType=1은 제목 플레이스홀더)
            if shape.Type == 14:  # msoPlaceholder
                if shape.PlaceholderFormat.Type == 1:  # ppPlaceholderTitle
                    # 텍스트 프레임이 있고 텍스트가 있는지 확인
                    if hasattr(shape, "TextFrame") and shape.TextFrame.HasText:
                        # 텍스트 색상을 파란색으로 변경
                        shape.TextFrame.TextRange.Font.Color.RGB = blue_color_rgb
                        print(f"슬라이드 {slide_index}의 제목 색상을 파란색으로 변경했습니다.")
    except Exception as e:
        print(f"슬라이드 {slide_index} 처리 중 오류 발생: {e}")

print("작업이 완료되었습니다.")

슬라이드 2의 제목 색상을 파란색으로 변경했습니다.
슬라이드 3의 제목 색상을 파란색으로 변경했습니다.
슬라이드 4의 제목 색상을 파란색으로 변경했습니다.
슬라이드 5의 제목 색상을 파란색으로 변경했습니다.
작업이 완료되었습니다.


## ppt 처음 부터 생성.
1. pptx_to_vba.py 로 TeXBLEU_ppt.pptx 파일에 대한 vba 코드를 추출함.
2. 해당 vba 코드를 입력으로 Claude에 이 ppt를 만들 수 있는 python 코드를 위 코드를 in-context 1-shot 으로 입력함.
3. 도형 위치 숫자 에러, 저장 에러 등 3번의 시도끝에 완전히 실행되는 코드를 얻을 수 있었음.

In [3]:
import win32com.client
import os
import tempfile
def create_texbleu_presentation(save_path="TeXBLEU_Presentation.pptx"):
    """
    TeXBLEU 프레젠테이션을 생성하는 함수
    
    Args:
        save_path (str): 저장할 파일 경로
    """
    # PowerPoint 애플리케이션 시작
    print("PowerPoint 애플리케이션 시작 중...")
    ppApp = win32com.client.Dispatch("PowerPoint.Application")
    ppApp.Visible = True
    
    # 프레젠테이션 생성
    ppPres = ppApp.Presentations.Add()
    
    # 슬라이드 크기 설정 (표준 와이드스크린 크기 사용)
    ppPres.PageSetup.SlideWidth = 960  # 표준 와이드스크린 프레젠테이션 너비 (points)
    ppPres.PageSetup.SlideHeight = 540  # 표준 와이드스크린 프레젠테이션 높이 (points)
    
    # PowerPoint 상수 정의
    ppLayoutBlank = 12
    ppLayoutText = 2
    ppLayoutTitle = 1
    ppLayoutTitleAndContent = 7
    msoTextOrientationHorizontal = 1
    msoShapeRectangle = 1  # 사각형 도형 상수
    
    print("슬라이드 생성 중...")
    
    # ----- 슬라이드 1 추가 (제목 슬라이드) -----
    ppSlide = ppPres.Slides.Add(1, ppLayoutTitle)
    
    # 텍스트 도형 추가: 제목
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=120, Top=90, 
                                    Width=720, Height=180)
    shp.TextFrame.TextRange.Text = "TeXBLEU: Automatic Metric for Evaluate LaTeX Format"
    
    # 텍스트 도형 추가: 부제목
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=200, Top=300, 
                                    Width=560, Height=150)
    shp.TextFrame.TextRange.Text = "Kyudan Jung1, Nam-Joon Kim2∗, Hyun Gon Ryu3, Sieun Hyeon2, Seung-jun Lee1, Hyuk-jae Lee2\n1Chung-Ang University, Seoul, Korea\n2Seoul National Univeristy, Seoul, Korea\n3NVIDIA"
    
    # 로고 이미지 자리에 사각형 추가 (이미지 대체)
    shp = ppSlide.Shapes.AddShape(msoShapeRectangle, 20, 450, 150, 75)  # 왼쪽 하단
    shp.Fill.ForeColor.RGB = 12632256  # 회색
    
    shp = ppSlide.Shapes.AddShape(msoShapeRectangle, 20, 20, 150, 75)  # 왼쪽 상단
    shp.Fill.ForeColor.RGB = 12632256  # 회색
    
    shp = ppSlide.Shapes.AddShape(msoShapeRectangle, 790, 20, 150, 75)  # 오른쪽 상단
    shp.Fill.ForeColor.RGB = 12632256  # 회색
    
    shp = ppSlide.Shapes.AddShape(msoShapeRectangle, 790, 450, 150, 75)  # 오른쪽 하단
    shp.Fill.ForeColor.RGB = 12632256  # 회색
    
    # 텍스트 도형 추가: 이메일
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=380, Top=490, 
                                    Width=200, Height=30)
    shp.TextFrame.TextRange.Text = "wjdrbeks1021@gmail.com"
    
    # ----- 슬라이드 2 추가 (개요 슬라이드) -----
    ppSlide = ppPres.Slides.Add(2, ppLayoutTitleAndContent)
    
    # 텍스트 도형 추가: 제목
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=80, Top=40, 
                                    Width=800, Height=100)
    shp.TextFrame.TextRange.Text = "Overview"
    
    # 텍스트 도형 추가: 내용
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=80, Top=150, 
                                    Width=800, Height=350)
    content_text = """Problem Define
1.1 Metric for LaTeX format
1.2 Limitations of existing metrics
Method
2.1 Tokenizer trained by arXiv papers
2.2 Fine-tuned embedding model
2.3 Computing TeXBLEU argorithm
Experiments
3.1 Dataset and setup
3.2 Human evaluation
3.3 Result
Conclusion"""
    shp.TextFrame.TextRange.Text = content_text
    
    # 텍스트 도형 추가: 슬라이드 번호
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=860, Top=500, 
                                    Width=80, Height=30)
    shp.TextFrame.TextRange.Text = "2/30"
    
    # ----- 슬라이드 3 추가 (Problem Define) -----
    ppSlide = ppPres.Slides.Add(3, ppLayoutTitleAndContent)
    
    # 텍스트 도형 추가: 제목
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=80, Top=40, 
                                    Width=800, Height=60)
    shp.TextFrame.TextRange.Text = "1.Problem Define"
    
    # 텍스트 도형 추가: 부제목
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=80, Top=90, 
                                    Width=800, Height=60)
    shp.TextFrame.TextRange.Text = "1.1 Metric for LaTeX Format"
    
    # 텍스트 도형 추가: 내용
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=80, Top=160, 
                                    Width=800, Height=320)
    content_text = """LaTeX format : Useful format in Mathematics, Computer Science, etc.
Example: In converting spoken mathematical expressions into LaTeX
Command-based language such as C or SQL
Suitable evaluation metric is required"""
    shp.TextFrame.TextRange.Text = content_text
    
    # 텍스트 도형 추가: 슬라이드 번호
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=860, Top=500, 
                                    Width=80, Height=30)
    shp.TextFrame.TextRange.Text = "3/30"
    
    # ----- 슬라이드 4 추가 (Limitations - BLEU) -----
    ppSlide = ppPres.Slides.Add(4, ppLayoutTitleAndContent)
    
    # 텍스트 도형 추가: 제목
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=80, Top=40, 
                                    Width=800, Height=60)
    shp.TextFrame.TextRange.Text = "1.Problem Define"
    
    # 텍스트 도형 추가: 부제목
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=80, Top=90, 
                                    Width=800, Height=60)
    shp.TextFrame.TextRange.Text = "1.2 Limitations of existing metrics"
    
    # 텍스트 도형 추가: 내용
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=80, Top=160, 
                                    Width=800, Height=60)
    shp.TextFrame.TextRange.Text = "BLEU"
    
    # 이미지 자리 표시자 (회색 사각형으로 대체)
    shp = ppSlide.Shapes.AddShape(msoShapeRectangle, 240, 240, 480, 200)
    shp.Fill.ForeColor.RGB = 12632256  # 회색
    
    # 텍스트 도형 추가: 슬라이드 번호
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=860, Top=500, 
                                    Width=80, Height=30)
    shp.TextFrame.TextRange.Text = "4/30"
    
    # ----- 슬라이드 5 추가 (Limitations - sacreBLEU) -----
    ppSlide = ppPres.Slides.Add(5, ppLayoutTitleAndContent)
    
    # 텍스트 도형 추가: 제목
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=80, Top=40, 
                                    Width=800, Height=60)
    shp.TextFrame.TextRange.Text = "1.Problem Define"
    
    # 텍스트 도형 추가: 부제목
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=80, Top=90, 
                                    Width=800, Height=60)
    shp.TextFrame.TextRange.Text = "1.2 Limitations of existing metrics"
    
    # 텍스트 도형 추가: 내용
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=80, Top=160, 
                                    Width=800, Height=60)
    shp.TextFrame.TextRange.Text = "sacreBLEU"
    
    # 이미지 자리 표시자 (회색 사각형으로 대체)
    shp = ppSlide.Shapes.AddShape(msoShapeRectangle, 240, 240, 480, 200)
    shp.Fill.ForeColor.RGB = 12632256  # 회색
    
    # 텍스트 도형 추가: 슬라이드 번호
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=860, Top=500, 
                                    Width=80, Height=30)
    shp.TextFrame.TextRange.Text = "5/30"
    
    # ----- 슬라이드 6 추가 (CER) -----
    ppSlide = ppPres.Slides.Add(6, ppLayoutTitleAndContent)
    
    # 텍스트 도형 추가: 제목
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=80, Top=40, 
                                    Width=800, Height=60)
    shp.TextFrame.TextRange.Text = "1.Problem Define"
    
    # 텍스트 도형 추가: 부제목
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=80, Top=90, 
                                    Width=800, Height=60)
    shp.TextFrame.TextRange.Text = "1.2 Limitations of existing metrics"
    
    # 텍스트 도형 추가: 내용
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=80, Top=160, 
                                    Width=800, Height=60)
    shp.TextFrame.TextRange.Text = "Character Error Rate (CER)"
    
    # 이미지 자리 표시자 (회색 사각형으로 대체)
    shp = ppSlide.Shapes.AddShape(msoShapeRectangle, 240, 240, 480, 200)
    shp.Fill.ForeColor.RGB = 12632256  # 회색
    
    # 텍스트 도형 추가: 슬라이드 번호
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=860, Top=500, 
                                    Width=80, Height=30)
    shp.TextFrame.TextRange.Text = "6/30"
    
    # ----- 슬라이드 7 추가 (WER) -----
    ppSlide = ppPres.Slides.Add(7, ppLayoutTitleAndContent)
    
    # 텍스트 도형 추가: 제목
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=80, Top=40, 
                                    Width=800, Height=60)
    shp.TextFrame.TextRange.Text = "1.Problem Define"
    
    # 텍스트 도형 추가: 부제목
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=80, Top=90, 
                                    Width=800, Height=60)
    shp.TextFrame.TextRange.Text = "1.2 Limitations of existing metrics"
    
    # 텍스트 도형 추가: 내용
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=80, Top=160, 
                                    Width=800, Height=60)
    shp.TextFrame.TextRange.Text = "Word Error Rate (WER)"
    
    # 이미지 자리 표시자 (회색 사각형으로 대체)
    shp = ppSlide.Shapes.AddShape(msoShapeRectangle, 240, 240, 480, 200)
    shp.Fill.ForeColor.RGB = 12632256  # 회색
    
    # 텍스트 도형 추가: 슬라이드 번호
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=860, Top=500, 
                                    Width=80, Height=30)
    shp.TextFrame.TextRange.Text = "7/30"
    
    # ----- 슬라이드 8 추가 (Method - Tokenizer) -----
    ppSlide = ppPres.Slides.Add(8, ppLayoutTitleAndContent)
    
    # 텍스트 도형 추가: 제목
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=80, Top=40, 
                                    Width=800, Height=60)
    shp.TextFrame.TextRange.Text = "2.Method"
    
    # 텍스트 도형 추가: 부제목
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=80, Top=90, 
                                    Width=800, Height=60)
    shp.TextFrame.TextRange.Text = "2.1 Tokenizer trained by arXiv papers"
    
    # 텍스트 도형 추가: 내용
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=80, Top=160, 
                                    Width=800, Height=320)
    content_text = """Requiring LaTeX-specified tokenizer
Collecting papers in arXiv .tex files
Byte-pair encoding (BPE) based tokenizer
Unlike other tokenizers, the tokenizer is designed to capture LaTeX grammar elements and tokenize them in a way that reflects LaTeX's unique structure"""
    shp.TextFrame.TextRange.Text = content_text
    
    # 텍스트 도형 추가: 슬라이드 번호
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=860, Top=500, 
                                    Width=80, Height=30)
    shp.TextFrame.TextRange.Text = "8/30"
    
    # ----- 슬라이드 9 추가 (Conclusion - 인덱스는 9지만 슬라이드 번호는 20/30으로 표시) -----
    ppSlide = ppPres.Slides.Add(9, ppLayoutTitleAndContent)
    
    # 텍스트 도형 추가: 제목
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=80, Top=40, 
                                    Width=800, Height=60)
    shp.TextFrame.TextRange.Text = "4.Conclusion"
    
    # 텍스트 도형 추가: 내용
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=80, Top=160, 
                                    Width=800, Height=320)
    content_text = """We proposed TeXBLEU, an automatic metric for evaluating the accuracy of LaTeX formats
TeXBLEU showed a significantly higher correlation with human evaluation metrics than other metrics
One limitation of this study is that there is no metric to verify whether the LaTeX format, when input into a compiler, can generate an error-free equation image.
This is a complex issue because different TeX command sets and compilers can result in different compile errors. Further research should address this issue."""
    shp.TextFrame.TextRange.Text = content_text
    
    # 텍스트 도형 추가: 슬라이드 번호
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=860, Top=500, 
                                    Width=80, Height=30)
    shp.TextFrame.TextRange.Text = "20/30"  # 슬라이드 인덱스는 9지만 번호는 20/30으로 표시
    
    # ----- 슬라이드 10 추가 (Thank You 슬라이드) -----
    ppSlide = ppPres.Slides.Add(10, ppLayoutTitle)
    
    # 텍스트 도형 추가: 제목
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=120, Top=90, 
                                    Width=720, Height=180)
    shp.TextFrame.TextRange.Text = "TeXBLEU: Automatic Metric for Evaluate LaTeX Format"
    
    # 텍스트 도형 추가: 부제목
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=200, Top=300, 
                                    Width=560, Height=150)
    shp.TextFrame.TextRange.Text = "Kyudan Jung1, Nam-Joon Kim2∗, Hyun Gon Ryu3, Sieun Hyeon2, Seung-jun Lee1, Hyuk-jae Lee2\n1Chung-Ang University, Seoul, Korea\n2Seoul National Univeristy, Seoul, Korea\n3NVIDIA"
    
    # 로고 이미지 자리에 사각형 추가 (이미지 대체)
    shp = ppSlide.Shapes.AddShape(msoShapeRectangle, 20, 450, 150, 75)  # 왼쪽 하단
    shp.Fill.ForeColor.RGB = 12632256  # 회색
    
    shp = ppSlide.Shapes.AddShape(msoShapeRectangle, 20, 20, 150, 75)  # 왼쪽 상단
    shp.Fill.ForeColor.RGB = 12632256  # 회색
    
    shp = ppSlide.Shapes.AddShape(msoShapeRectangle, 790, 20, 150, 75)  # 오른쪽 상단
    shp.Fill.ForeColor.RGB = 12632256  # 회색
    
    shp = ppSlide.Shapes.AddShape(msoShapeRectangle, 790, 450, 150, 75)  # 오른쪽 하단
    shp.Fill.ForeColor.RGB = 12632256  # 회색
    
    # 텍스트 도형 추가: 이메일 및 발표자 정보
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=380, Top=480, 
                                    Width=200, Height=50)
    shp.TextFrame.TextRange.Text = "Presenter: Kyudan Jung\nwjdrbeks1021@gmail.com"
    
    # 텍스트 도형 추가: Thank you
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=400, Top=50, 
                                    Width=200, Height=50)
    shp.TextFrame.TextRange.Text = "Thank you"
    
    try:
        print("프레젠테이션 저장 중...")
        
        # 임시 디렉토리에 저장 경로 설정
        temp_dir = tempfile.gettempdir()
        temp_filename = f"TeXBLEU_Presentation_{int(time.time())}.pptx"
        save_path = os.path.join(temp_dir, temp_filename)
        
        # 프레젠테이션 저장
        ppPres.SaveAs(save_path)
        print(f"프레젠테이션이 다음 위치에 저장되었습니다: {save_path}")
        
        return ppPres, save_path
    
    except Exception as e:
        print(f"저장 중 오류 발생: {str(e)}")
        
        # 대안으로 사용자에게 직접 저장하도록 안내
        print("PowerPoint 애플리케이션이 열려 있습니다. 직접 [파일 > 다른 이름으로 저장]을 선택하여 저장하세요.")
        
        return ppPres, None

if __name__ == "__main__":
    # 실행 경로 설정
    save_path = os.path.join(os.path.expanduser("~"), "Desktop", "TeXBLEU_Presentation.pptx")
    
    # 프레젠테이션 생성
    prs = create_texbleu_presentation(save_path)
    
    print(f"프레젠테이션이 다음 위치에 저장되었습니다: {save_path}")

PowerPoint 애플리케이션 시작 중...
슬라이드 생성 중...
프레젠테이션 저장 중...
프레젠테이션이 다음 위치에 저장되었습니다: C:\Users\Public\Documents\ESTsoft\CreatorTemp\TeXBLEU_Presentation_1743949890.pptx
프레젠테이션이 다음 위치에 저장되었습니다: C:\Users\wjdrb\Desktop\TeXBLEU_Presentation.pptx


## delay 주면서 생성.

In [7]:
import win32com.client
import os
import tempfile
import time  # time 모듈 추가
import win32gui  # 윈도우 조작을 위한 모듈 추가

def create_texbleu_presentation(save_path="TeXBLEU_Presentation.pptx", delay=1.0, highlight_editing=True):
    """
    TeXBLEU 프레젠테이션을 생성하는 함수
    
    Args:
        save_path (str): 저장할 파일 경로
        delay (float): 각 단계 사이의 딜레이 시간(초)
        highlight_editing (bool): 편집 중인 요소를 강조 표시할지 여부
    """
    # PowerPoint 애플리케이션 시작
    print("PowerPoint 애플리케이션 시작 중...")
    ppApp = win32com.client.Dispatch("PowerPoint.Application")
    ppApp.Visible = True
    
    # PowerPoint 창을 최대화
    ppApp.WindowState = 2  # msoWindowStateMaximized
    
    # PowerPoint 창을 활성화
    def bring_powerpoint_to_front():
        try:
            # PowerPoint 창의 핸들 찾기
            hwnd = win32gui.FindWindow("PPTFrameClass", None)
            if hwnd:
                # 창이 최소화되어 있다면 복원
                if win32gui.IsIconic(hwnd):
                    win32gui.ShowWindow(hwnd, 9)  # SW_RESTORE
                # 창을 맨 앞으로 가져오기
                win32gui.SetForegroundWindow(hwnd)
                return True
        except Exception as e:
            print(f"창 활성화 중 오류: {str(e)}")
        return False
    
    bring_powerpoint_to_front()
    time.sleep(delay)  # 딜레이 추가
    
    # 프레젠테이션 생성
    print("프레젠테이션 생성 중...")
    ppPres = ppApp.Presentations.Add()
    time.sleep(delay)  # 딜레이 추가
    
    # 슬라이드 크기 설정 (표준 와이드스크린 크기 사용)
    print("슬라이드 크기 설정 중...")
    ppPres.PageSetup.SlideWidth = 960  # 표준 와이드스크린 프레젠테이션 너비 (points)
    ppPres.PageSetup.SlideHeight = 540  # 표준 와이드스크린 프레젠테이션 높이 (points)
    time.sleep(delay)  # 딜레이 추가
    
    # PowerPoint 상수 정의
    ppLayoutBlank = 12
    ppLayoutText = 2
    ppLayoutTitle = 1
    ppLayoutTitleAndContent = 7
    msoTextOrientationHorizontal = 1
    msoShapeRectangle = 1  # 사각형 도형 상수
    
    print("슬라이드 생성 시작...")
    
    # 편집 중인 요소를 시각적으로 표시하는 함수
    def show_editing(slide, msg):
        """편집 중인 작업을 화면에 표시하는 함수"""
        if not highlight_editing:
            return
            
        # 슬라이드로 이동하고 슬라이드 쇼 뷰를 사용하지 않음
        slide.Select()
        bring_powerpoint_to_front()
        
        # PowerPoint 상태 표시줄에 메시지 표시 (일부 PowerPoint 버전에서만 작동)
        try:
            ppApp.StatusBar = f"편집 중: {msg}"
        except:
            pass
    
    # ----- 슬라이드 1 추가 (제목 슬라이드) -----
    print("슬라이드 1 생성 중: 제목 슬라이드")
    ppSlide = ppPres.Slides.Add(1, ppLayoutTitle)
    show_editing(ppSlide, "슬라이드 1 - 제목 슬라이드")
    time.sleep(delay)  # 딜레이 추가
    
    # 텍스트 도형 추가: 제목
    print("  제목 추가 중...")
    #shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
    #                                Left=120, Top=90, 
    #                                Width=720, Height=180)
    
    shp = ppSlide.Shapes.Title  # 기존 제목 개체 가져오기
    shp.TextFrame.TextRange.Text = "TeXBLEU: Automatic Metric for Evaluate LaTeX Format"
    shp.TextFrame.TextRange.Font.Bold = True
    shp.TextFrame.TextRange.Font.Size = 60
    shp.TextFrame.TextRange.ParagraphFormat.Alignment = 2    
    time.sleep(delay/2)  # 더 짧은 딜레이
    
    # 텍스트 도형 추가: 부제목
    print("  부제목 추가 중...")
    #shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
    #                                Left=200, Top=300, 
    #                                Width=560, Height=150)
    shp = ppSlide.Shapes.Placeholders(2)  #
    shp.TextFrame.TextRange.Text = "Kyudan Jung1, Nam-Joon Kim2∗, Hyun Gon Ryu3, Sieun Hyeon2, Seung-jun Lee1, Hyuk-jae Lee2\n1Chung-Ang University, Seoul, Korea\n2Seoul National Univeristy, Seoul, Korea\n3NVIDIA"
    
    
    time.sleep(delay/2)  # 더 짧은 딜레이

    # 로고 이미지 자리에 사각형 추가 (이미지 대체)
    print("  로고 자리 표시자 추가 중...")
    
    # 각 로고 위치에 대한 설명
    logo_positions = [
        {"desc": "왼쪽 하단 로고", "left": 20, "top": 450, "width": 150, "height": 75},
        {"desc": "왼쪽 상단 로고", "left": 20, "top": 20, "width": 150, "height": 75},
        {"desc": "오른쪽 상단 로고", "left": 790, "top": 20, "width": 150, "height": 75},
        {"desc": "오른쪽 하단 로고", "left": 790, "top": 450, "width": 150, "height": 75}
    ]
    
    # 각 로고 위치에 사각형 추가하고 시각적으로 표시
    for pos in logo_positions:
        print(f"    {pos['desc']} 추가 중...")
        shp = ppSlide.Shapes.AddShape(msoShapeRectangle, pos['left'], pos['top'], pos['width'], pos['height'])
        shp.Fill.ForeColor.RGB = 12632256  # 회색
        
        # 모양 선택하여 편집 중임을 보여줌
        try:
            shp.Select()
            show_editing(ppSlide, pos['desc'])
            # 모양 강조
            if highlight_editing:
                original_color = shp.Fill.ForeColor.RGB
                shp.Fill.ForeColor.RGB = 5287936  # 밝은 녹색
                time.sleep(delay/4)
                shp.Fill.ForeColor.RGB = original_color
        except:
            pass
            
        time.sleep(delay/8)  # 짧은 딜레이
    
    time.sleep(delay/2)  # 더 짧은 딜레이
    
    # 텍스트 도형 추가: 이메일
    print("  이메일 추가 중...")
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=380, Top=490, 
                                    Width=200, Height=30)
    shp.TextFrame.TextRange.Text = "wjdrbeks1021@gmail.com"
    time.sleep(delay/2)
    # 텍스트가 한줄로 펴지게.
    # 이게 일관적으로 적용되려면 슬라이드 크기보다 작게 텍스트박스가 확장되는 코드가 필요함.
    shp.TextFrame2.WordWrap = False

    #import sys
    #sys.exit()
        
    time.sleep(delay)  # 딜레이 추가
    
    # ----- 슬라이드 2 추가 (개요 슬라이드) -----
    print("슬라이드 2 생성 중: 개요 슬라이드")
    ppSlide = ppPres.Slides.Add(2, ppLayoutTitleAndContent)
    show_editing(ppSlide, "슬라이드 2 - 개요 슬라이드")
    time.sleep(delay)  # 딜레이 추가
    
    # 텍스트 도형 추가: 제목
    print("  제목 추가 중...")
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=80, Top=40, 
                                    Width=800, Height=100)
    shp.TextFrame.TextRange.Text = "Overview"
    time.sleep(delay/2)  # 더 짧은 딜레이
    
    # 텍스트 도형 추가: 내용
    print("  내용 추가 중...")
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=80, Top=150, 
                                    Width=800, Height=350)
    content_text = """Problem Define
1.1 Metric for LaTeX format
1.2 Limitations of existing metrics
Method
2.1 Tokenizer trained by arXiv papers
2.2 Fine-tuned embedding model
2.3 Computing TeXBLEU argorithm
Experiments
3.1 Dataset and setup
3.2 Human evaluation
3.3 Result
Conclusion"""
    shp.TextFrame.TextRange.Text = content_text
    shp.TextFrame.TextRange.Font.Size = 20
    time.sleep(delay/2)  # 더 짧은 딜레이
    
    # 텍스트 도형 추가: 슬라이드 번호
    print("  슬라이드 번호 추가 중...")
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=860, Top=500, 
                                    Width=80, Height=30)
    shp.TextFrame.TextRange.Text = "2/30"
    time.sleep(delay)  # 딜레이 추가
    
    # 나머지 슬라이드에 대한 함수
    def create_slide_with_delay(slide_num, layout, title_text, subtitle_text=None, content_text=None, page_num=None, image_placeholders=None):
        print(f"슬라이드 {slide_num} 생성 중: {title_text}")
        ppSlide = ppPres.Slides.Add(slide_num, layout)
        show_editing(ppSlide, f"슬라이드 {slide_num} - {title_text}")
        time.sleep(delay)  # 딜레이 추가
        
        # 제목 추가
        print(f"  제목 추가 중: {title_text}")
        shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                        Left=80, Top=40, 
                                        Width=800, Height=60)
        shp.TextFrame.TextRange.Text = title_text
        
        # 제목 텍스트 강조
        try:
            shp.Select()
            show_editing(ppSlide, f"제목 텍스트 편집: {title_text}")
            if highlight_editing:
                shp.TextFrame.TextRange.Font.Bold = True
                time.sleep(delay/4)
                shp.TextFrame.TextRange.Font.Bold = False
        except:
            pass
            
        time.sleep(delay/3)  # 더 짧은 딜레이
        
        # 부제목이 있을 경우 추가
        if subtitle_text:
            print(f"  부제목 추가 중: {subtitle_text}")
            shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                            Left=80, Top=90, 
                                            Width=800, Height=60)
            shp.TextFrame.TextRange.Text = subtitle_text
            
            # 부제목 텍스트 강조
            try:
                shp.Select()
                show_editing(ppSlide, f"부제목 편집: {subtitle_text}")
                if highlight_editing:
                    shp.TextFrame.TextRange.Font.Italic = True
                    time.sleep(delay/4)
                    shp.TextFrame.TextRange.Font.Italic = False
            except:
                pass
                
            time.sleep(delay/3)  # 더 짧은 딜레이
        
        # 내용이 있을 경우 추가
        if content_text:
            print("  내용 추가 중...")
            shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                            Left=80, Top=160, 
                                            Width=800, Height=320)
            shp.TextFrame.TextRange.Text = content_text
            
            # 내용 텍스트 강조
            try:
                shp.Select()
                show_editing(ppSlide, "내용 텍스트 편집")
                if highlight_editing:
                    shp.TextFrame.TextRange.Font.Color.RGB = 7647264  # 진한 파란색
                    time.sleep(delay/4)
                    shp.TextFrame.TextRange.Font.Color.RGB = 0  # 검은색
            except:
                pass
                
            time.sleep(delay/3)  # 더 짧은 딜레이
        
        # 이미지 자리 표시자가 있을 경우 추가
        if image_placeholders:
            print("  이미지 자리 표시자 추가 중...")
            for i, placeholder in enumerate(image_placeholders):
                shp = ppSlide.Shapes.AddShape(msoShapeRectangle, 
                                              placeholder['left'], 
                                              placeholder['top'], 
                                              placeholder['width'], 
                                              placeholder['height'])
                shp.Fill.ForeColor.RGB = 12632256  # 회색
                
                # 이미지 자리 표시자 강조
                try:
                    shp.Select()
                    show_editing(ppSlide, f"이미지 자리 표시자 {i+1} 추가")
                    if highlight_editing:
                        original_color = shp.Fill.ForeColor.RGB
                        shp.Fill.ForeColor.RGB = 10485760  # 파란색
                        time.sleep(delay/4)
                        shp.Fill.ForeColor.RGB = original_color
                except:
                    pass
                    
                time.sleep(delay/6)  # 더 짧은 딜레이
                
            time.sleep(delay/6)  # 더 짧은 딜레이
        
        # 슬라이드 번호가 있을 경우 추가
        if page_num:
            print(f"  슬라이드 번호 추가 중: {page_num}")
            shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                            Left=860, Top=500, 
                                            Width=80, Height=30)
            shp.TextFrame.TextRange.Text = page_num
            
            # 슬라이드 번호 강조
            try:
                shp.Select()
                show_editing(ppSlide, f"슬라이드 번호 추가: {page_num}")
                if highlight_editing:
                    shp.TextFrame.TextRange.Font.Bold = True
                    time.sleep(delay/4)
                    shp.TextFrame.TextRange.Font.Bold = False
            except:
                pass
                
            time.sleep(delay/3)  # 더 짧은 딜레이
        
        print(f"슬라이드 {slide_num} 완성")
        # 전체 슬라이드 보기
        try:
            ppSlide.Select()
            time.sleep(delay/2)
        except:
            pass
            
        time.sleep(delay/2)  # 더 짧은 딜레이
        
        return ppSlide
    
    # 이미지 자리 표시자 정의
    standard_image_placeholder = [{'left': 240, 'top': 240, 'width': 480, 'height': 200}]
    
    # ----- 슬라이드 3 추가 (Problem Define) -----
    create_slide_with_delay(
        3, 
        ppLayoutTitleAndContent, 
        "1.Problem Define", 
        "1.1 Metric for LaTeX Format", 
        """LaTeX format : Useful format in Mathematics, Computer Science, etc.
Example: In converting spoken mathematical expressions into LaTeX
Command-based language such as C or SQL
Suitable evaluation metric is required""", 
        "3/30"
    )
    
    # ----- 슬라이드 4 추가 (Limitations - BLEU) -----
    create_slide_with_delay(
        4, 
        ppLayoutTitleAndContent, 
        "1.Problem Define", 
        "1.2 Limitations of existing metrics", 
        "BLEU", 
        "4/30", 
        standard_image_placeholder
    )
    
    # ----- 슬라이드 5 추가 (Limitations - sacreBLEU) -----
    create_slide_with_delay(
        5, 
        ppLayoutTitleAndContent, 
        "1.Problem Define", 
        "1.2 Limitations of existing metrics", 
        "sacreBLEU", 
        "5/30", 
        standard_image_placeholder
    )
    
    # ----- 슬라이드 6 추가 (CER) -----
    create_slide_with_delay(
        6, 
        ppLayoutTitleAndContent, 
        "1.Problem Define", 
        "1.2 Limitations of existing metrics", 
        "Character Error Rate (CER)", 
        "6/30", 
        standard_image_placeholder
    )
    
    # ----- 슬라이드 7 추가 (WER) -----
    create_slide_with_delay(
        7, 
        ppLayoutTitleAndContent, 
        "1.Problem Define", 
        "1.2 Limitations of existing metrics", 
        "Word Error Rate (WER)", 
        "7/30", 
        standard_image_placeholder
    )
    
    # ----- 슬라이드 8 추가 (Method - Tokenizer) -----
    create_slide_with_delay(
        8, 
        ppLayoutTitleAndContent, 
        "2.Method", 
        "2.1 Tokenizer trained by arXiv papers", 
        """Requiring LaTeX-specified tokenizer
Collecting papers in arXiv .tex files
Byte-pair encoding (BPE) based tokenizer
Unlike other tokenizers, the tokenizer is designed to capture LaTeX grammar elements and tokenize them in a way that reflects LaTeX's unique structure""", 
        "8/30"
    )
    
    # ----- 슬라이드 9 추가 (Conclusion - 인덱스는 9지만 슬라이드 번호는 20/30으로 표시) -----
    create_slide_with_delay(
        9, 
        ppLayoutTitleAndContent, 
        "4.Conclusion", 
        None, 
        """We proposed TeXBLEU, an automatic metric for evaluating the accuracy of LaTeX formats
TeXBLEU showed a significantly higher correlation with human evaluation metrics than other metrics
One limitation of this study is that there is no metric to verify whether the LaTeX format, when input into a compiler, can generate an error-free equation image.
This is a complex issue because different TeX command sets and compilers can result in different compile errors. Further research should address this issue.""", 
        "20/30"
    )
    
    # ----- 슬라이드 10 추가 (Thank You 슬라이드) -----
    print("슬라이드 10 생성 중: Thank You 슬라이드")
    ppSlide = ppPres.Slides.Add(10, ppLayoutTitle)
    show_editing(ppSlide, "슬라이드 10 - Thank You 슬라이드")
    time.sleep(delay)  # 딜레이 추가
    
    # 텍스트 도형 추가: 제목
    print("  제목 추가 중...")
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=120, Top=90, 
                                    Width=720, Height=180)
    shp.TextFrame.TextRange.Text = "TeXBLEU: Automatic Metric for Evaluate LaTeX Format"
    
    # 텍스트 선택하여 편집 중임을 보여줌
    try:
        shp.Select()
        show_editing(ppSlide, "제목 텍스트 편집")
        # 텍스트에 강조 효과 적용
        if highlight_editing:
            shp.TextFrame.TextRange.Font.Bold = True
            time.sleep(delay/4)
            shp.TextFrame.TextRange.Font.Bold = False
    except:
        pass
        
    time.sleep(delay/2)  # 더 짧은 딜레이
    
    # 텍스트 도형 추가: Thank you
    print("  Thank you 메시지 추가 중...")
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=400, Top=50, 
                                    Width=200, Height=50)
    shp.TextFrame.TextRange.Text = "Thank you"
    
    # 텍스트 선택하여 편집 중임을 보여줌
    try:
        shp.Select()
        show_editing(ppSlide, "Thank you 메시지 편집")
        # 텍스트에 강조 효과 적용
        if highlight_editing:
            # 글꼴 크기 증가
            original_size = shp.TextFrame.TextRange.Font.Size
            shp.TextFrame.TextRange.Font.Size = original_size * 1.5
            shp.TextFrame.TextRange.Font.Bold = True
            shp.TextFrame.TextRange.Font.Color.RGB = 5287936  # 녹색
            time.sleep(delay/2)
            shp.TextFrame.TextRange.Font.Size = original_size
            shp.TextFrame.TextRange.Font.Bold = False
            shp.TextFrame.TextRange.Font.Color.RGB = 0  # 검은색
    except:
        pass
        
    time.sleep(delay/2)  # 더 짧은 딜레이
    
    # 텍스트 도형 추가: 부제목
    print("  저자 정보 추가 중...")
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=200, Top=300, 
                                    Width=560, Height=150)
    shp.TextFrame.TextRange.Text = "Kyudan Jung1, Nam-Joon Kim2∗, Hyun Gon Ryu3, Sieun Hyeon2, Seung-jun Lee1, Hyuk-jae Lee2\n1Chung-Ang University, Seoul, Korea\n2Seoul National Univeristy, Seoul, Korea\n3NVIDIA"
    
    # 텍스트 선택하여 편집 중임을 보여줌
    try:
        shp.Select()
        show_editing(ppSlide, "저자 정보 편집")
        # 텍스트에 강조 효과 적용
        if highlight_editing:
            shp.TextFrame.TextRange.Font.Italic = True
            time.sleep(delay/4)
            shp.TextFrame.TextRange.Font.Italic = False
    except:
        pass
        
    time.sleep(delay/2)  # 더 짧은 딜레이
    
    # 로고 이미지 자리에 사각형 추가 (이미지 대체)
    print("  로고 자리 표시자 추가 중...")
    
    # 각 로고 위치에 대한 설명
    logo_positions = [
        {"desc": "왼쪽 하단 로고", "left": 20, "top": 450, "width": 150, "height": 75},
        {"desc": "왼쪽 상단 로고", "left": 20, "top": 20, "width": 150, "height": 75},
        {"desc": "오른쪽 상단 로고", "left": 790, "top": 20, "width": 150, "height": 75},
        {"desc": "오른쪽 하단 로고", "left": 790, "top": 450, "width": 150, "height": 75}
    ]
    
    # 각 로고 위치에 사각형 추가하고 시각적으로 표시
    for pos in logo_positions:
        print(f"    {pos['desc']} 추가 중...")
        shp = ppSlide.Shapes.AddShape(msoShapeRectangle, pos['left'], pos['top'], pos['width'], pos['height'])
        shp.Fill.ForeColor.RGB = 12632256  # 회색
        
        # 모양 선택하여 편집 중임을 보여줌
        try:
            shp.Select()
            show_editing(ppSlide, pos['desc'])
            # 모양 강조
            if highlight_editing:
                original_color = shp.Fill.ForeColor.RGB
                shp.Fill.ForeColor.RGB = 5287936  # 밝은 녹색
                time.sleep(delay/4)
                shp.Fill.ForeColor.RGB = original_color
        except:
            pass
            
        time.sleep(delay/8)  # 짧은 딜레이
    
    time.sleep(delay/2)  # 더 짧은 딜레이
    
    # 텍스트 도형 추가: 이메일 및 발표자 정보
    print("  발표자 정보 추가 중...")
    shp = ppSlide.Shapes.AddTextbox(Orientation=msoTextOrientationHorizontal, 
                                    Left=380, Top=480, 
                                    Width=200, Height=50)
    shp.TextFrame.TextRange.Text = "Presenter: Kyudan Jung\nwjdrbeks1021@gmail.com"
    
    # 텍스트 선택하여 편집 중임을 보여줌
    try:
        shp.Select()
        show_editing(ppSlide, "발표자 정보 및 이메일 편집")
        # 텍스트에 강조 효과 적용
        if highlight_editing:
            shp.TextFrame.TextRange.Font.Underline = True
            time.sleep(delay/4)
            shp.TextFrame.TextRange.Font.Underline = False
    except:
        pass
        
    time.sleep(delay)  # 딜레이 추가
    
    try:
        print("프레젠테이션 저장 준비 중...")
        time.sleep(delay)  # 딜레이 추가
        
        # 모든 슬라이드 보기 모드로 전환
        try:
            ppApp.ActiveWindow.ViewType = 9  # ppViewSlideSorter
            print("슬라이드 정렬 보기로 전환합니다...")
            time.sleep(delay*2)  # 더 긴 딜레이
            
            # 일반 보기로 돌아가기
            ppApp.ActiveWindow.ViewType = 1  # ppViewNormal
            print("일반 보기로 돌아갑니다...")
            time.sleep(delay)
            
            # 첫 번째 슬라이드 선택
            ppPres.Slides(1).Select()
            print("첫 번째 슬라이드로 이동합니다...")
            time.sleep(delay)
        except:
            print("슬라이드 보기 모드 전환 중 오류가 발생했습니다.")
        
        # 임시 디렉토리에 저장 경로 설정
        temp_dir = tempfile.gettempdir()
        temp_filename = f"TeXBLEU_Presentation_{int(time.time())}.pptx"
        actual_save_path = os.path.join(temp_dir, temp_filename)
        
        # 프레젠테이션 저장
        print(f"프레젠테이션을 다음 위치에 저장 중: {actual_save_path}")
        ppPres.SaveAs(actual_save_path)
        time.sleep(delay)  # 딜레이 추가
        print(f"프레젠테이션이 성공적으로 저장되었습니다: {actual_save_path}")
        
        return ppPres, actual_save_path
    
    except Exception as e:
        print(f"저장 중 오류 발생: {str(e)}")
        time.sleep(delay)  # 딜레이 추가
        
        # 대안으로 사용자에게 직접 저장하도록 안내
        print("PowerPoint 애플리케이션이 열려 있습니다. 직접 [파일 > 다른 이름으로 저장]을 선택하여 저장하세요.")
        
        return ppPres, None

if __name__ == "__main__":
    # 실행 경로 설정
    save_path = os.path.join(os.path.expanduser("~"), "Desktop", "TeXBLEU_Presentation.pptx")
    
    # 딜레이 값을 지정하여 프레젠테이션 생성 (초 단위)
    delay_time = 1.0  # 기본 딜레이를 1초로 설정
    
    # 편집 중인 슬라이드 시각적으로 보여주기 여부
    highlight_editing = True
    
    print(f"===== TeXBLEU 프레젠테이션 생성 시작 (딜레이: {delay_time}초) =====")
    print("참고: 딜레이 시간을 조정하려면 함수 호출 시 delay 매개변수를 변경하세요.")
    print(f"현재 편집 중인 요소 시각적 표시: {'활성화됨' if highlight_editing else '비활성화됨'}")
    time.sleep(delay_time)  # 시작 전 딜레이
    
    # 프레젠테이션 생성
    prs, saved_path = create_texbleu_presentation(save_path, delay=delay_time, highlight_editing=highlight_editing)
    
    if saved_path:
        print(f"프레젠테이션이 다음 위치에 저장되었습니다: {saved_path}")
    else:
        print("프레젠테이션이 자동으로 저장되지 않았습니다. 수동으로 저장해주세요.")
    
    print("===== TeXBLEU 프레젠테이션 생성 완료 =====")

===== TeXBLEU 프레젠테이션 생성 시작 (딜레이: 1.0초) =====
참고: 딜레이 시간을 조정하려면 함수 호출 시 delay 매개변수를 변경하세요.
현재 편집 중인 요소 시각적 표시: 활성화됨
PowerPoint 애플리케이션 시작 중...
프레젠테이션 생성 중...
슬라이드 크기 설정 중...
슬라이드 생성 시작...
슬라이드 1 생성 중: 제목 슬라이드
  제목 추가 중...
  부제목 추가 중...
  로고 자리 표시자 추가 중...
    왼쪽 하단 로고 추가 중...
    왼쪽 상단 로고 추가 중...
    오른쪽 상단 로고 추가 중...
    오른쪽 하단 로고 추가 중...
  이메일 추가 중...
슬라이드 2 생성 중: 개요 슬라이드
  제목 추가 중...
  내용 추가 중...
  슬라이드 번호 추가 중...
슬라이드 3 생성 중: 1.Problem Define
  제목 추가 중: 1.Problem Define
  부제목 추가 중: 1.1 Metric for LaTeX Format
  내용 추가 중...
  슬라이드 번호 추가 중: 3/30
슬라이드 3 완성
슬라이드 4 생성 중: 1.Problem Define
  제목 추가 중: 1.Problem Define
  부제목 추가 중: 1.2 Limitations of existing metrics
  내용 추가 중...
  이미지 자리 표시자 추가 중...
  슬라이드 번호 추가 중: 4/30
슬라이드 4 완성
슬라이드 5 생성 중: 1.Problem Define
  제목 추가 중: 1.Problem Define
  부제목 추가 중: 1.2 Limitations of existing metrics
  내용 추가 중...
  이미지 자리 표시자 추가 중...
  슬라이드 번호 추가 중: 5/30
슬라이드 5 완성
슬라이드 6 생성 중: 1.Problem Define
  제목 추가 중: 1.Problem Define
  부제목 추가 중: 1.2 Limitations of existing

In [9]:
import win32com.client as win32

def get_slide_title(ppt_path, slide_number=8):
    """
    PowerPoint 파일에서 특정 슬라이드의 제목을 추출하는 함수
    
    Parameters:
        ppt_path (str): PowerPoint 파일의 경로
        slide_number (int): 제목을 추출할 슬라이드 번호 (기본값: 8)
        
    Returns:
        str: 슬라이드 제목 (제목이 없는 경우 "제목 없음" 반환)
    """
    # PowerPoint 애플리케이션 객체 생성
    ppt_app = win32.Dispatch("PowerPoint.Application")
    
    try:
        # Visible 속성 설정 시도 (실패해도 계속 진행)
        try:
            ppt_app.Visible = False
        except:
            pass  # Visible 속성 설정 실패 시 무시하고 계속 진행
        
        # 프레젠테이션 열기
        presentation = ppt_app.Presentations.Open(ppt_path)
        
        # 슬라이드 번호 확인 (유효한 범위인지)
        if slide_number <= 0 or slide_number > presentation.Slides.Count:
            return f"오류: 유효하지 않은 슬라이드 번호입니다. 슬라이드는 1부터 {presentation.Slides.Count}까지 있습니다."
            
        # 지정된 슬라이드 가져오기
        slide = presentation.Slides[slide_number]
        
        # 슬라이드에서 제목 모양 찾기
        title = None
        for shape in slide.Shapes:
            if shape.HasTextFrame:
                if shape.TextFrame.HasText:
                    # 제목 플레이스홀더인지 확인 시도
                    try:
                        if shape.PlaceholderFormat.Type == 1:  # 1은 제목 플레이스홀더 타입
                            title = shape.TextFrame.TextRange.Text
                            break
                    except:
                        # PlaceholderFormat이 없는 경우 계속 진행
                        pass
        
        # 제목을 찾지 못했다면 전체 슬라이드에서 텍스트 검색
        if title is None:
            for shape in slide.Shapes:
                if shape.HasTextFrame and shape.TextFrame.HasText:
                    # 첫 번째 텍스트 프레임을 제목으로 간주
                    title = shape.TextFrame.TextRange.Text
                    break
            
        # 프레젠테이션 닫기 (저장하지 않음)
        presentation.Close()
        
        return title if title else "제목 없음"
        
    except Exception as e:
        return f"오류 발생: {str(e)}"
        
    finally:
        # PowerPoint 애플리케이션 종료
        try:
            ppt_app.Quit()
        except:
            pass  # 종료 실패 시 무시

if __name__ == "__main__":
    # 파일 경로 설정 (실제 PPT 파일 경로로 변경 필요)
    ppt_file_path = r"TeXBLEU_ppt.pptx"
    
    # 8번 슬라이드의 제목 가져오기
    slide_title = get_slide_title(ppt_file_path)
    
    print(f"8번 슬라이드 제목: {slide_title}")

8번 슬라이드 제목: 2.Method
